In [5]:
import pandas as pd
import glob
import json
import geopandas
import geocoder
import folium
import json
import ast
import re
# import cartopy.crs as ccrs
import matplotlib.pyplot as plt

from geopy.geocoders import Nominatim
from shapely.geometry import Polygon, LineString, Point
import math
import numpy as np

In [6]:
offers_df = pd.read_csv('all_data\oferta_eksport_final.csv')
pracuj_df = pd.read_csv('all_data\pracuj_inferred_location.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'all_data\\oferta_eksport_final.csv'

### Mergowanie datasetów

In [5]:
offers_df.head()

NameError: name 'offers_df' is not defined

In [6]:
pracuj_df.head()

NameError: name 'pracuj_df' is not defined

### Praca w polsce - markery

In [14]:
offers_df_country_markers = offers_df.filter(['open_street_raw'], axis=1)

In [15]:
offers_df_country_markers['open_street_raw'] = offers_df_country_markers.apply(lambda row: ast.literal_eval(row['open_street_raw']), axis=1)

In [16]:
offers_df_country_markers['marker_coords'] = \
    offers_df_country_markers.apply(lambda row: [float(row['open_street_raw']['lat']), float(row['open_street_raw']['lon'])], axis=1)

In [88]:
pracuj_df_country_markers = pracuj_df.filter(['inferred_location'], axis=1)

In [89]:
pracuj_df_country_markers = pracuj_df_country_markers.dropna().copy()

In [90]:
pracuj_df_country_markers['inferred_location'] = pracuj_df_country_markers.apply(lambda row: ast.literal_eval(row['inferred_location']), axis=1)

In [94]:
pracuj_df_country_markers['marker_coords'] = \
    pracuj_df_country_markers.apply(lambda row: [float(row['inferred_location']['y']), float(row['inferred_location']['x'])], axis=1)

In [98]:
df_country_markers = pd.concat([offers_df_country_markers, pracuj_df_country_markers], ignore_index=True)

In [20]:
START_LAT = 51.8512195
START_LON = 20.8417665
START_ZOOM = 6

In [99]:
map_country_markers = folium.Map(location=[START_LAT, START_LON], zoom_start=START_ZOOM)

In [100]:
for index, row in df_country_markers.iterrows():
    folium.CircleMarker(
        location=row['marker_coords'],
        tooltip=row['marker_coords'],
        radius=2,
#         weight=1,
        color="crimson",
        fill=False,
    ).add_to(map_country_markers)

In [101]:
map_country_markers.save("map_country_markers.html")

### Dalsza obróbka datasetu z markerami

In [102]:
# df_markers = df_country_markers.copy()

In [110]:
# geolocator = Nominatim(user_agent="http")
# df_markers['geopy_address'] = \
#     df_markers.apply(lambda row: geolocator.reverse(row['marker_coords']).address.split(', '), axis=1)

In [7]:
# df_wroclaw_markers.to_csv('markers_with_adressess.csv', index=True)
df_markers = pd.read_csv('all_data/markers_with_adressess.csv')

In [8]:
df_markers.drop_duplicates(subset=['open_street_raw', 'marker_coords', 'inferred_location', 'geopy_address'], inplace=True)

In [9]:
df_markers['geopy_address'] = df_markers.apply(lambda row: ast.literal_eval(row['geopy_address']), axis=1)
df_markers['marker_coords'] = df_markers.apply(lambda row: ast.literal_eval(row['marker_coords']), axis=1)

In [10]:
df_markers['geopy_address_len'] = df_markers.apply(lambda row: len(row['geopy_address']), axis=1)

In [11]:
df_markers.drop(df_markers[df_markers['geopy_address_len'] <= 4].index, inplace=True)
df_markers.drop(['geopy_address_len'], inplace = True, axis=1)
df_markers.drop(['Unnamed: 0'], inplace = True, axis=1)

In [12]:
df_markers['country'] = df_markers.apply(lambda row: row['geopy_address'][-1], axis=1)
df_markers.drop(df_markers[df_markers['country'] != 'Polska'].index, inplace=True)
df_markers.drop(['country'], inplace = True, axis=1)

In [13]:
df_markers['voivodeship'] = df_markers.apply(lambda row: row['geopy_address'][-3], axis=1)

In [14]:
df_markers['state'] = df_markers.apply(lambda row: row['geopy_address'][-4], axis=1)

In [15]:
df_markers['county'] = df_markers.apply(lambda row: row['geopy_address'][-5], axis=1)

### Praca we wrocławiu - markery

In [16]:
df_wroclaw_markers = df_markers[df_markers['state'] == 'Wrocław'].copy()

In [17]:
df_wroclaw_markers['county'] = df_wroclaw_markers.apply(lambda row: row['county'].replace("Osiedle ", "", 1), axis=1)

In [18]:
df_wroclaw_markers['count'] = df_wroclaw_markers.apply(lambda row: 1, axis=1)

In [19]:
df_wroclaw_markers_count = df_wroclaw_markers.groupby(['county'])['count'].sum().reset_index()

In [17]:
df_wroclaw_markers_count['county'] = df_wroclaw_markers_count.apply(lambda row: row['county'].replace("-", " - "), axis=1)

In [30]:
df_wroclaw_markers_count['count_log'] = np.log10(df_wroclaw_markers_count['count'])

In [31]:
df_wroclaw_markers_count

,county,count,county_log,count_log
0,Biskupin-Sępolno-Dąbie-Bartoszowice,11,2.397895,1.041393
1,Borek,9,2.197225,0.954243
2,Brochów,1,0.000000,0.000000
3,Gaj,5,1.609438,0.698970
4,Gajowice,4,1.386294,0.602060
5,Grabiszyn-Grabiszynek,11,2.397895,1.041393
6,Gądów-Popowice Południowe,4,1.386294,0.602060
7,Huby,4,1.386294,0.602060
8,Jagodno,3,1.098612,0.477121
9,Jerzmanowo-Jarnołtów-Strachowice-Osiniec,8,2.079442,0.903090


In [21]:
df_wroclaw_markers_count.drop(df_wroclaw_markers_count[df_wroclaw_markers_count['county'] == 'Stare Miasto'].index, inplace=True)

In [22]:
df_wroclaw_markers_count['count_norm'] = \
    (df_wroclaw_markers_count['count']-df_wroclaw_markers_count['count'].min())/(df_wroclaw_markers_count['count'].max()-df_wroclaw_markers_count['count'].min())

In [34]:
WROC_START_LAT = 51.13694038154205 
WROC_START_LON = 17.04576517482033
WROC_START_ZOOM = 11

wroclaw_geo = geopandas.read_file('geo_maps/miasta/wroclaw-max.geojson')

wroclaw_map = folium.Map(location=[WROC_START_LAT, WROC_START_LON], zoom_start=WROC_START_ZOOM)

folium.Choropleth(
    geo_data=wroclaw_geo,
    name="choropleth",
    data=df_wroclaw_markers_count,
    columns=["county", "count_log"],
    key_on="feature.properties.osiedle",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Jobs count",
).add_to(wroclaw_map)

folium.LayerControl().add_to(wroclaw_map)

for index, row in df_wroclaw_markers.iterrows():
    folium.CircleMarker(
        location=row['marker_coords'],
        tooltip=row['marker_coords'],
        radius=2,
#         weight=1,
        color="crimson",
        fill=False,
    ).add_to(wroclaw_map)


In [35]:
wroclaw_map

In [391]:
wroclaw_map.save("map_wroclaw_markers.html")

### Liczba ogłoszeń o pracę - województwa

In [11]:
df_voivodeship_markers = df_markers.copy()

In [12]:
df_voivodeship_markers.drop(df_voivodeship_markers[~df_voivodeship_markers['voivodeship'].str.contains('województwo')].index, inplace=True)

In [13]:
df_voivodeship_markers['voivodeship'] = df_voivodeship_markers.apply(lambda row: row['voivodeship'].replace("województwo ", "", 1), axis=1)

In [14]:
df_voivodeship_markers['count'] = df_voivodeship_markers.apply(lambda row: 1, axis=1)

In [61]:
df_voivodeship_markers_count = df_voivodeship_markers.groupby(['voivodeship'])['count'].sum().reset_index()

In [64]:
df_voivodeship_markers_count['count_log'] = np.log(df_voivodeship_markers_count['count'])

In [21]:
voivodeship_geo = geopandas.read_file('geo_maps/wojewodztwa/wojewodztwa-max.geojson')

voivodeship_map = folium.Map(location=[START_LAT, START_LON], zoom_start=START_ZOOM)

folium.Choropleth(
    geo_data=voivodeship_geo,
    name="choropleth",
    data=df_voivodeship_markers_count,
    columns=["voivodeship", "count"],
    key_on="feature.properties.nazwa",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Jobs count",
).add_to(voivodeship_map)

folium.LayerControl().add_to(voivodeship_map)

for index, row in df_voivodeship_markers.iterrows():
    folium.CircleMarker(
        location=row['marker_coords'],
        tooltip=row['marker_coords'],
        radius=0.5,
#         weight=1,
        color="crimson",
        fill=False,
    ).add_to(voivodeship_map)

In [22]:
voivodeship_map.save("map_voivodeship_markers.html")

### Liczba ogłoszeń o pracę - powiaty

In [23]:
df_state_markers = df_markers.copy()

In [24]:
df_state_markers.drop(df_state_markers[~df_state_markers['voivodeship'].str.contains('województwo')].index, inplace=True)

In [26]:
state_geo = geopandas.read_file('geo_maps/powiaty/powiaty-max.geojson')

In [27]:
def get_state_name(row):
    current_point = Point(row['marker_coords'][1], row['marker_coords'][0])
    containing_series = state_geo.contains(current_point)
    containing_series_bools = containing_series[containing_series == True]
    
    if containing_series_bools.empty:
        state = 'None'
    else:
        state = state_geo[['id', 'nazwa']].iloc[containing_series_bools.index.item()]['nazwa']
    return state

In [28]:
current_point = Point(18.994487141198455, 50.25972211320963)
current_point = Point(17.847767227493087, 49.68307698890241)


d = state_geo.contains(current_point)

In [29]:
df_state_markers['state'] = df_state_markers.apply(lambda row: get_state_name(row), axis=1)

In [30]:
df_state_markers['count'] = df_state_markers.apply(lambda row: 1, axis=1)

In [31]:
df_state_markers[df_state_markers['state'] == 'None']

,open_street_raw,marker_coords,inferred_location,geopy_address,voivodeship,state,county,count
37526,NaN,"[54.61025975, 18.514110303450295]","{'x': 18.514110303450295, 'y': 54.61025975, 'a...","[Molo w Mechelinkach, Nadmorska, Mechelinki, g...",województwo pomorskie,None,gmina Kosakowo,1


In [40]:
df_state_markers_count = df_state_markers.groupby(['state'])['count'].sum().reset_index()

In [42]:
df_state_markers_count["count_log"] = np.log(df_state_markers_count["count"])

In [43]:
df_state_markers_count.drop(df_state_markers_count[df_state_markers_count['state'] == 'None'].index, inplace=True)

In [44]:
df_state_markers_count.drop(df_state_markers_count[df_state_markers_count['state'] == 'powiat Warszawa'].index, inplace=True)

In [65]:
# state_geo = geopandas.read_file('D:\Studia\Semestr IX\Przetwarzanie Danych Przestrzennych\ms_A02_Granice_powiatow.gml')
# state_geo = geopandas.read_file('D:\Studia\Semestr IX\Przetwarzanie Danych Przestrzennych\geo_maps\powiaty\powiaty-max.geojson')

state_map = folium.Map(location=[START_LAT, START_LON], zoom_start=START_ZOOM)

folium.Choropleth(
    geo_data=state_geo,
    name="Jobs count per district",
    data=df_state_markers_count,
    columns=["state", "count_log"],
    key_on="feature.properties.nazwa",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Jobs count per district (log scale)",
).add_to(state_map)


folium.Choropleth(
    geo_data=voivodeship_geo,
    name="Jobs count per voivodeship",
    data=df_voivodeship_markers_count,
    columns=["voivodeship", "count_log"],
    key_on="feature.properties.nazwa",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Jobs count per voivodeship (log scale)",
).add_to(state_map)


circlesLayer = folium.FeatureGroup(name="Job offers").add_to(state_map)

for index, row in df_state_markers.iterrows():
    folium.CircleMarker(
        location=row['marker_coords'],
        tooltip=row['marker_coords'],
        radius=0.5,
#         weight=1,
        color="crimson",
        fill=False,
    ).add_to(circlesLayer)

folium.LayerControl().add_to(state_map)

In [66]:
state_map.save("map_state_voivodeship_markers.html")

### Bezrobocie - polska

In [108]:
df_unemployment_country = pd.read_pickle('all_data/df_bezrobocie_polska.pkl')
df_unemployment_voivodeship = pd.read_pickle('all_data/df_bezrobocie_wojew.pkl')
df_unemployment_state = pd.read_pickle('all_data/df_bezrobocie_powiat.pkl')

In [109]:
df_unemployment_voivodeship['unit'] = df_unemployment_voivodeship.apply(lambda row: row['unit'].replace(" ", ""), axis=1)

In [110]:
df_unemployment_voivodeship['unit'] = df_unemployment_voivodeship.apply(lambda row: row['unit'].replace("Woj.", "", 1).lower(), axis=1)

In [111]:
df_unemployment_state['unit'] = df_unemployment_state.apply(lambda row: row['unit'].replace(" ", ""), axis=1)

In [112]:
LANGS = [
    'ĄąĆćĘęŁłŃńÓóŚśŹźŻż',               # Polish
    ]
LANGS_ = [
    'ĄĆĘŁŃÓŚŹŻ',               # Polish
    ]

pattern = '[A-Za-z{langs}][^A-Z{langs_}]*'.format(langs=''.join(LANGS), langs_=''.join(LANGS_))
pattern = re.compile(pattern)

df_unemployment_state['unit'] = \
    df_unemployment_state.apply(lambda row: ' '.join(pattern.findall(row['unit'])), axis=1)

In [113]:
df_unemployment_state['unit'] = df_unemployment_state.apply(lambda row: 'powiat ' + row['unit'], axis=1)

In [114]:
df_unemployment_state['unit'] = df_unemployment_state.apply(lambda row: row['unit'].replace("m. ", "", 1), axis=1)
df_unemployment_state['unit'] = df_unemployment_state.apply(lambda row: row['unit'].replace("m.", "", 1), axis=1)

In [115]:
df_unemployment_state['unit'] = df_unemployment_state.apply(lambda row: row['unit'].replace("warszawskizachodni", "warszawski zachodni", 1), axis=1)
df_unemployment_state['unit'] = df_unemployment_state.apply(lambda row: row['unit'].replace("łódzkiwschodni", "łódzki wschodni", 1), axis=1)
df_unemployment_state['unit'] = df_unemployment_state.apply(lambda row: row['unit'].replace("karkonoski", "jeleniogórski", 1), axis=1)
df_unemployment_state['unit'] = df_unemployment_state.apply(lambda row: row['unit'].replace("Jastrzębie- Zdrój", "Jastrzębie-Zdrój", 1), axis=1)
df_unemployment_state['unit'] = df_unemployment_state.apply(lambda row: row['unit'].replace("Bielsko- Biała", "Bielsko-Biała", 1), axis=1)

In [116]:
df_unemployment_state[df_unemployment_state['unit'].str.contains('Biel')]['unit']

283    powiat Bielsko-Biała
Name: unit, dtype: object

### Bezrobocie - województwa

In [117]:
voivodeship_geo = geopandas.read_file('geo_maps/wojewodztwa/wojewodztwa-max.geojson')

voivodeship_map = folium.Map(location=[START_LAT, START_LON], zoom_start=START_ZOOM)

folium.Choropleth(
    geo_data=voivodeship_geo,
    name="choropleth",
    data=df_unemployment_voivodeship,
    columns=["unit", "rate_p"],
    key_on="feature.properties.nazwa",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Unemployment rate",
).add_to(voivodeship_map)

folium.LayerControl().add_to(voivodeship_map)

In [121]:
voivodeship_map.save("map_voivodeship_unemployment.html")

### Bezrobocie - powiaty

In [125]:
state_geo = geopandas.read_file('geo_maps/powiaty/powiaty-max.geojson')

state_map = folium.Map(location=[START_LAT, START_LON], zoom_start=START_ZOOM)

folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=df_unemployment_state,
    columns=["unit", "rate_p"],
    key_on="feature.properties.nazwa",
    fill_color="Reds",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Unemployment rate",
).add_to(state_map)

folium.LayerControl().add_to(state_map)

In [127]:
state_map.save("map_state_unemployment.html")

# Badanie odległości od najbliższej oferty pracy

In [25]:
df_markers = pd.read_csv("preprocessed_markers_with_adressess.csv")

In [80]:
state_geo

,id,nazwa,geometry
0,1,powiat ropczycko-sędziszowski,"MULTIPOLYGON (((21.69430 50.17850, 21.69440 50..."
1,2,powiat łosicki,"MULTIPOLYGON (((22.72380 52.28040, 22.72380 52..."
2,3,powiat piaseczyński,"MULTIPOLYGON (((21.11010 52.11320, 21.10880 52..."
3,4,powiat radomski,"MULTIPOLYGON (((20.98910 51.48950, 20.98920 51..."
4,5,powiat sierpecki,"MULTIPOLYGON (((19.50880 52.86780, 19.50870 52..."
...,...,...,...
375,376,powiat Żory,"MULTIPOLYGON (((18.64180 50.06880, 18.64170 50..."
376,377,powiat nowodworski,"MULTIPOLYGON (((19.33020 54.36650, 19.33140 54..."
377,378,powiat jaworski,"MULTIPOLYGON (((16.19850 51.01200, 16.19850 51..."
378,379,powiat poznański,"MULTIPOLYGON (((16.60050 52.41220, 16.60040 52..."


# ----------------PATRYK-----------------------

In [130]:
import geopandas
from shapely.geometry import Polygon
import numpy as np
from matplotlib.cm import rainbow
from hexalattice.hexalattice import create_hex_grid
import math
import pandas as pd
import glob
import json
import geopandas
import geocoder
import folium
import json
import ast
import re
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
import pickle

START_LAT = 51.8512195
START_LON = 20.8417665
START_ZOOM = 6
state_geo = geopandas.read_file('geo_maps/powiaty/powiaty-max.geojson')
df_markers = pd.read_csv('markers_with_adressess.csv')


xmin, ymin, xmax, ymax = state_geo.total_bounds

In [131]:
def create_hexagon(l, x, y):
    """
    Create a hexagon centered on (x, y)
    :param l: length of the hexagon's edge
    :param x: x-coordinate of the hexagon's center
    :param y: y-coordinate of the hexagon's center
    :return: The polygon containing the hexagon's coordinates
    """
    c = [[x + math.cos(math.radians(angle)) * l, y + math.sin(math.radians(angle)) * l] for angle in range(0, 360, 60)]
    return Polygon(c)

def create_hexgrid(bbox, side):
    """
    returns an array of Points describing hexagons centers that are inside the given bounding_box
    :param bbox: The containing bounding box. The bbox coordinate should be in Webmercator.
    :param side: The size of the hexagons'
    :return: The hexagon grid
    """
    grid = []

    v_step = math.sqrt(3) * side
    h_step = 1.5 * side

    x_min = min(bbox[0], bbox[2])
    x_max = max(bbox[0], bbox[2])
    y_min = min(bbox[1], bbox[3])
    y_max = max(bbox[1], bbox[3])

    h_skip = math.ceil(x_min / h_step) - 1
    h_start = h_skip * h_step

    v_skip = math.ceil(y_min / v_step) - 1
    v_start = v_skip * v_step

    h_end = x_max + h_step
    v_end = y_max + v_step

    if v_start - (v_step / 2.0) < y_min:
        v_start_array = [v_start + (v_step / 2.0), v_start]
    else:
        v_start_array = [v_start - (v_step / 2.0), v_start]

    v_start_idx = int(abs(h_skip) % 2)

    c_x = h_start
    c_y = v_start_array[v_start_idx]
    v_start_idx = (v_start_idx + 1) % 2
    while c_x < h_end:
        while c_y < v_end:
            grid.append((c_x, c_y))
            c_y += v_step
        c_x += h_step
        c_y = v_start_array[v_start_idx]
        v_start_idx = (v_start_idx + 1) % 2

    return grid


In [132]:
offers_coords = [ast.literal_eval(marker) for marker in df_markers["marker_coords"]]
nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(offers_coords)

In [141]:
# # calculate polygons
RESOLUTION=0.05

edge = math.sqrt(RESOLUTION**2/(3/2 * math.sqrt(3)))
hex_centers = create_hexgrid(state_geo.total_bounds, edge)

polygons = []
polygons_centers = []

# print("Creating hexes")
for center in tqdm(hex_centers):
    poly = create_hexagon(edge, center[0], center[1])
    centroid = poly.centroid
    if any(state_geo.contains(centroid)):
        polygons.append(poly)
        polygons_centers.append(centroid.coords[0][::-1])


with open("my_pkl/polygons.pkl", "wb") as file:
    pickle.dump(polygons, file)

with open("my_pkl/polygons.pkl", "rb") as file:
    polygons = pickle.load(file)

    
with open("my_pkl/polygons_centers.pkl", "wb") as file:
    pickle.dump(polygons_centers, file)

with open("my_pkl/polygons_centers.pkl", "rb") as file:
    polygons_centers = pickle.load(file)

# find nearest offer for each poly
nearest = nbrs.kneighbors(polygons_centers, n_neighbors=1)
nearest = [item[0] for item in nearest[0]]

with open("my_pkl/nearest.pkl", "wb") as file:
    pickle.dump(nearest, file)
    
with open("my_pkl/nearest.pkl", "rb") as file:
    nearest = pickle.load(file)
    

# create grid and distances in grid
grid = geopandas.GeoDataFrame({'geometry':polygons, 'grid_id':list(range(len(polygons)))}, crs="EPSG:4326")
df_grid = pd.DataFrame({"grid_id": list(range(len(polygons))), "dist": nearest})

state_map = folium.Map(location=[START_LAT, START_LON], zoom_start=START_ZOOM)


folium.Choropleth(
    geo_data=state_geo,
    name="unemplyoment",
    data=df_unemployment_state,
    columns=["unit", "rate_p"],
    key_on="feature.properties.nazwa",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Unemployment rate",
).add_to(state_map)

# folium.LayerControl().add_to(state_map)


folium.Choropleth(
    geo_data=grid,
    name="distance to job",
    data=df_grid,
    columns=["grid_id", "dist"],
    key_on="feature.properties.grid_id",
    fill_color="Reds",
    fill_opacity=0.5,
    line_opacity=0.00,
    legend_name="Distance to closest job",
).add_to(state_map)

folium.LayerControl().add_to(state_map)

state_map.save("distance_closest_job_simp.html")

100%|████████████████████████████████████| 23870/23870 [00:33<00:00, 702.09it/s]
